<a href="https://colab.research.google.com/github/talhaanwarch/OffenseEval2020/blob/master/Arabic/sub/sub1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Arabic Language

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd /content/drive/My Drive/dataset/OffenseEval2020/data/Arabic/

/content/drive/My Drive/dataset/OffenseEval2020/data/Arabic


In [3]:
ls

cc.ar.300.vec             offenseval-ar-test-tweets-v1.tsv
cc.ar.300.vec.gz          offenseval-ar-training-v1.tsv
offenseval-ar-dev-v1.tsv  readme-data-ar.txt


In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
train=pd.read_csv( 'offenseval-ar-training-v1.tsv',sep="\t")
train=train.sample(frac=1)

In [5]:
train.head()

,id,tweet,subtask_a
4421,4493,@USER @USER يسلم فمك يا ست الستات يا بولا يا ح...,NOT
4263,4335,يا جماعة ويلفريد زاها يا جماعة!!! 🔥,NOT
2159,2166,RT @USER: من لا يشجعه وقت انكساره لا يستحق يشج...,NOT
3598,3670,يا صباح الأنوار يا حلمٍ حصلي <LF>يا حبيبٍ هيض ...,NOT
6402,6545,فينك يا علولي<LF> فينك يا مروان محسن <LF> فينك...,NOT


In [0]:
train["tweet"]= train["tweet"].str.replace('RT', "") 
train["tweet"]= train["tweet"].str.replace('@USER', "") 
train['tweet']=train['tweet'].str.replace('\d+', '')
train['tweet']=train['tweet'].str.replace('URL', '')


In [0]:
train_labels=train['subtask_a']
X_train=train['tweet']
y_train=le.fit_transform(train_labels)

In [8]:
y_train[0:5]

array([0, 0, 0, 0, 0])

In [0]:
val=pd.read_csv( 'offenseval-ar-dev-v1.tsv',sep="\t")

In [0]:
val["tweet"]= val["tweet"].str.replace('RT', "") 
val["tweet"]= val["tweet"].str.replace('@USER', "") 
val['tweet']=val['tweet'].str.replace('\d+', '')
val['tweet']=val['tweet'].str.replace('URL', '')


In [11]:
import collections
collections.Counter(train_labels)

Counter({'NOT': 5468, 'OFF': 1371})

In [0]:
val_labels=val['subtask_a']
X_val=val['tweet']
y_val=le.fit_transform(val_labels)

In [15]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
max_words = 10000 #frequency of words to be kept
max_len = 200

tokenize = Tokenizer(num_words=max_words)
tokenize.fit_on_texts(X_train)
sequences = tokenize.texts_to_sequences(X_train)
word_index = tokenize.word_index
X_train_sequences = sequence.pad_sequences(sequences,maxlen=max_len)

Using TensorFlow backend.


In [16]:
len(X_train_sequences),len(train_labels)

(6839, 6839)

In [0]:

val_sequences = tokenize.texts_to_sequences(X_val)
X_val_sequences = sequence.pad_sequences(val_sequences,maxlen=max_len)

In [0]:
from keras.models import Sequential
from keras.layers import Embedding,CuDNNGRU,GRU,Dense,Dropout,Bidirectional,GlobalMaxPool1D,GlobalAveragePooling1D, SpatialDropout1D
from keras.optimizers import RMSprop
from keras.initializers import Constant
from sklearn.utils import class_weight
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [0]:
from keras import backend as K
def f1(y_true, y_pred):
    '''
    metric from here 
    https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
    '''
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ar.300.vec.gz
# import gzip
# import shutil
# with gzip.open('cc.ar.300.vec.gz', 'rb') as f_in:
#     with open('cc.ar.300.vec', 'wb') as f_out:
#         shutil.copyfileobj(f_in, f_out)

In [21]:
num_words = min(max_words, len(word_index)) + 1
print(num_words)

10001


In [0]:
embed_size=300
def get_coefs(word,*arr):
    return word, np.asarray(arr, dtype='float32')

def build_matrix(embedding_path, word_index):
    embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path))

    nb_words = min(max_words, len(word_index))
    embedding_matrix = np.zeros((nb_words + 1, embed_size))
    for word, i in word_index.items():
        if i >= max_words:
            continue
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

embeddings=build_matrix('cc.ar.300.vec', word_index)


In [0]:
model = Sequential()
model.add(Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False))
model.add(Bidirectional(CuDNNGRU(150, return_sequences = True)))
#model.add(Bidirectional(CuDNNGRU(50, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(50, activation="relu"))
model.add(Dense(20, activation="relu"))

model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1])

In [0]:
import numpy as np
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
class_weights=dict(enumerate(class_weights))

In [0]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.000001, verbose=1)

In [26]:

model.fit(X_train_sequences,y_train,batch_size=64,epochs=10,verbose=2,class_weight=class_weights,validation_data=(X_val_sequences,y_val),callbacks=[reduce_lr])

Train on 6839 samples, validate on 1000 samples
Epoch 1/10
 - 20s - loss: 0.5116 - f1: 0.5434 - val_loss: 0.3691 - val_f1: 0.6466
Epoch 2/10
 - 6s - loss: 0.3356 - f1: 0.7049 - val_loss: 0.3192 - val_f1: 0.6772
Epoch 3/10
 - 6s - loss: 0.2909 - f1: 0.7382 - val_loss: 0.2872 - val_f1: 0.7041
Epoch 4/10
 - 6s - loss: 0.2459 - f1: 0.7796 - val_loss: 0.2506 - val_f1: 0.7215
Epoch 5/10
 - 6s - loss: 0.2024 - f1: 0.8209 - val_loss: 0.2511 - val_f1: 0.7219
Epoch 6/10
 - 6s - loss: 0.1537 - f1: 0.8650 - val_loss: 0.2907 - val_f1: 0.7324

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 7/10
 - 6s - loss: 0.0948 - f1: 0.9283 - val_loss: 0.2803 - val_f1: 0.7316
Epoch 8/10
 - 6s - loss: 0.0799 - f1: 0.9414 - val_loss: 0.2843 - val_f1: 0.7219

Epoch 00008: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
Epoch 9/10
 - 6s - loss: 0.0652 - f1: 0.9500 - val_loss: 0.2895 - val_f1: 0.7174
Epoch 10/10
 - 6s - loss: 0.0632 - f1: 0.9578 - val_loss: 

In [27]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_val_sequences, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
y_pred = (y_pred > 0.5)

print(classification_report(y_val, y_pred))

1000/1000 [==============================] - 0s 317us/step
              precision    recall  f1-score   support

           0       0.95      0.93      0.94       821
           1       0.70      0.77      0.73       179

    accuracy                           0.90      1000
   macro avg       0.82      0.85      0.83      1000
weighted avg       0.90      0.90      0.90      1000



In [28]:
test=pd.read_csv( 'offenseval-ar-test-tweets-v1.tsv',sep="\t")
test["tweet"]= test["tweet"].str.replace('RT', "") 
test["tweet"]= test["tweet"].str.replace('@USER', "") 
test['tweet']=test['tweet'].str.replace('\d+', '')
test['tweet']=test['tweet'].str.replace('URL', '')

test.head()

,id,tweet
0,8001,اما انت تقعد طول عمرك لا مبدا ولا راي ثابت يا...
1,8002,بتخاف نسوانك يزعلوا ولا ايه 😂 اه يا هلفوت يا...
2,8003,: يا عـسانـى نـبـقى يا عـمري حـبايـب وحـبنـا ...
3,8004,: باقي البيان وينو ما شفنه يا برهان <LF>ورينا...
4,8005,اللهم انت الشافي المعافي اشفيه وجميع مرضى ال...


In [0]:
X_test=test['tweet']

test_sequences = tokenize.texts_to_sequences(X_test)
X_test_sequences = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [52]:
y_test = model.predict(X_test_sequences, batch_size=32, verbose=1)


1827/1827 [==============================] - 1s 402us/step


In [0]:
y_test[y_test>0.5]=1
y_test[y_test<0.5]=0
y_test=y_test.astype('int16').ravel()

In [0]:
y_test=le.inverse_transform(y_test)

In [0]:
y_test=pd.DataFrame(y_test,columns=['label'])

In [57]:
y_test.head()

,label
0,OFF
1,OFF
2,NOT
3,OFF
4,NOT


In [0]:
y_test=pd.concat([test['id'], y_test['label']], axis=1)


In [59]:
y_test.head()

,id,label
0,8001,OFF
1,8002,OFF
2,8003,NOT
3,8004,OFF
4,8005,NOT


In [0]:
#NOT zero
#OFF one

In [0]:
y_test.to_csv('/content/sub1.csv',index=False,header=None)


In [0]:
# import keras
# keras.backend.clear_session()